# Exploration of Unlucky Strike

In [7]:
import base64
import serv
import os
from Crypto.Random import get_random_bytes
from Crypto.Util.number import long_to_bytes
from Crypto.Cipher import AES

In [2]:
A = serv.Server()
serv.Nballs = 5
A.numbers = [int.from_bytes(get_random_bytes(3), "big")   for i in range(serv.Nballs)]

In [197]:
A.numbers

[13967147, 526602, 8804009, 16256181, 15433614]

In [80]:
ticket = base64.b64decode(A.getTicket())
print(len(rtb)-16, rtb)

b'numbers:5451,50199,35627,36972,2345\r\r\r\r\r\r\r\r\r\r\r\r\r'
80 b"7Dc\xf4\xbb\x92G\xd0\xce2\x80S\xc8\x15\xd3\x91\xf3\xdf\x17\xb2\xe0\xb9\x86\x8c>\xd18\xbb\xa7\x16\x99\xc0\xae\xef\x06ab\x19w\x80\x90\x18\xa9\xd7=4\x0e\xe3|Q\xee\xe0P\x08\xe0\xf0\x98\x7f\xd8\x8f\x8a.\xff\x1e\xe3s\xe6\xac\xa5Y\xd6\x15\xb4t\x9aHm_\xc5\xcb@B3\xa0g\x1bs4>\xebr'\xd5\x9c\x95\x0b"


In [34]:
#trytick = b'w\x8c\xc7l4\xfe\xcf\xddP\x7f\x81\x1c\x00\xe4\xab\x84\x05\x8d\xad\x00\xa7\x1c_\xa46\xef\xdax\x80\xc2g\xd0,\xcd\x19\xabO\xee\x9a\xb7\xd7\x0f\xdb[\x06\xcd"\xce\xf6\x9d\xa8\xf1\xbe\xb1w\xbc|\xdb\xe0\x945\xfe\x83\xd4'
dec = AES.new(A.key, AES.MODE_CBC, trytick[:16]).decrypt(trytick[16:])
print(dec)
print(dec[-18])

b'numbers:17950,55\xabZ\xb3\x9a(\x08\x1ej\x8e\x08\xeb\x18\x9cwx945\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x02\x02'
120


In [156]:
# Padding Oracle Attack

C_blc = []
D_blc = []
P_blc = []

for blc in range(3):
    
    rtb = ticket[:len(ticket)-16*blc]
    #print(rtb)
    
    C = b''
    D = b''
    P = b''

    for byt in range(1,17):

        #print(byt)
        assert len(C) == len(D) == len(P)

        og_byt = rtb[-16 - byt]

        pb = bytes([D[i] ^ byt for i in range(len(D))][::-1])
        padbyt = rtb[:(len(rtb)-16)-(byt-1)] + pb + rtb[-16:]

        assert len(padbyt) == len(rtb)

        b_correct = None
        pos_lst = []
        for b in range(256):

            trytick = bytearray( padbyt )

            trytick[-16 - byt] = b

            assert len(trytick) == len(rtb)

            check = A.redeemTicket(base64.b64encode( ticket + trytick ))

            if check:
                pos_lst.append(b)

        if len(pos_lst) == 1:
            b_correct = pos_lst[0]
        elif len(pos_lst) == 2:
            b1,b2 = pos_lst
            if b1 == padbyt[-16 - byt]:
                b_correct = b2
            if b2 == padbyt[-16 - byt]:
                b_correct = b1
        else:
            print(pos_lst)
            raise ValueError('Oh oh...')

        #print(pos_lst)            
        C += long_to_bytes(b_correct)
        D += long_to_bytes(b_correct ^ byt)
        P += long_to_bytes(b_correct ^ byt ^ og_byt)
        
        #print(P[::-1])
    
    C_blc.append(C[::-1])
    D_blc.append(D[::-1])
    P_blc.append(P[::-1])

print(b''.join(P_blc[::-1]))

b'numbers:5451,50199,35627,36972,2345\r\r\r\r\r\r\r\r\r\r\r\r\r'


In [233]:
# Padding Oracle Attack

payload = b'numbers:' + ','.join([str(i) for i in A.numbers]).encode()
padpay = payload + (16-len(payload)%16) * long_to_bytes(16-len(payload)%16)
print(padpay)
print()

C_blc = []
D_blc = []

tick_pl = b'\xaa'*16*5 #+ ticket[-16:]
print(tick_pl)

for blc in range(4):
    
    rtb = tick_pl[:len(tick_pl)-16*blc]
    #print(rtb)
    
    D = b''

    for byt in range(1,17):

        pb = bytes([D[i] ^ byt for i in range(len(D))][::-1])
        padbyt = rtb[:(len(rtb)-16)-(byt-1)] + pb + rtb[-16:]

        assert len(padbyt) == len(rtb)

        b_correct = None
        pos_lst = []
        for b in range(256):

            trytick = bytearray( padbyt )

            trytick[-16 - byt] = b

            assert len(trytick) == len(rtb)

            check = A.redeemTicket(base64.b64encode( ticket + trytick ))

            if check:
                pos_lst.append(b)
                break

        if len(pos_lst) == 1:
            b_correct = pos_lst[0]
        elif len(pos_lst) == 2:
            b1,b2 = pos_lst
            if b1 == padbyt[-16 - byt]:
                b_correct = b2
            if b2 == padbyt[-16 - byt]:
                b_correct = b1
        else:
            print(pos_lst)
            raise ValueError('Oh oh...')
          
        D += long_to_bytes(b_correct ^ byt)
        
    pl_blc = padpay[len(padpay)-16*(blc+1):len(padpay)-16*blc]
    
    C_rep = bytes([pl_blc[i] ^ D[-(i+1)] for i in range(16)])
    
    tick_pl = tick_pl[:len(padpay)-16*(blc+1)] + C_rep + tick_pl[len(padpay)-16*blc:]
    
    D_blc.append(D[::-1])

print(b''.join(D_blc[::-1]))
print()
print(tick_pl)

b'numbers:13967147,526602,8804009,16256181,15433614\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'

b'\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa'
b'"y\xa4\x92\xcd{&F\xe53#|\xc0O\xd7\xe4\xb0\xf5\xf8\x0e{v\xe9$K\xd5s\xc1\x0f\xb5\xe1[\xa8\xed=]\x84\x02\xf8\x9d\x0c\x89e\x99\x88{Z\xd1\x10\x03\xc5\xad\x87\xfbM\n\x00p\xd8\xc6e?\xba\xa7'

b'L\x0c\xc9\xf0\xa8\tU|\xd4\x00\x1aJ\xf7~\xe3\xd3\x9c\xc0\xca8MF\xdb\x08s\xedC\xf5?\x85\xd8w\x99\xdb\x0fh\xb23\xc0\xac \xb8P\xad\xbbHl\xe0$\x0c\xca\xa2\x88\xf4B\x05\x0f\x7f\xd7\xc9j0\xb5\xa8\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa\xaa'


In [234]:
A.redeemTicket(base64.b64encode( tick_pl ))

**JACKPOT**  -> Here is your reward: NONO


True

In [229]:
ticket

b'\xea\x84<K\x04\xec\x9e\x82p\x1b\xfa\xbex/\x88\xe5iv \xa0M~\x8f>w\xdd\xb1\xbe\x8f(\x10.\x95\x12Z\x1b\xa9\xf5\x01\xd7K\x99\xe0\xdd\xa9M\x16\x82\xae\x94\xfeS\xc3\xce\xd7\x85\xc5\xa3\xe8Q\x9be\xccr'

In [223]:
def unpkcs7(padded):
    while True:
        if (len(padded) & 0xf) != 0 or len(padded) == 0: break
        padlen = padded[-1]
        if padlen < 1 or padlen > 16: break
        if any([ padded[-(i + 1)] != padlen for i in range(padlen)]): break
        return padded[:(-padlen)]
    raise ValueError

def redeemTicket(key, ticket):
    try:
        tenc = base64.b64decode(ticket)
        cipher = AES.new(key, AES.MODE_CBC, tenc[:16])
        traw = cipher.decrypt(tenc[16:])
        print(traw)
        traw = unpkcs7(traw)
        print(traw)
        nums = [v for v in traw[traw.index(b"numbers")+8:].split(b",")] [:5]
        print(nums)
    except (ValueError, IndexError):
        #print("that is an invalid ticket")
        return False
    if any([ str(v).encode("ascii") not in nums for v in A.numbers]):
        print(f"sorry, that ticket did not win anything")
        print(cipher.decrypt(tenc[16:]))
        return True
    else:
        print(f"**JACKPOT**  -> Here is your reward: {FLAG}")
        return True

In [224]:
redeemTicket(A.key, base64.b64encode( tick_pl ))

b'numbers:13967147, 526602, 8804009, 16256181, 15433614\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'
b'numbers:13967147, 526602, 8804009, 16256181, 15433614'
[b'13967147', b' 526602', b' 8804009', b' 16256181', b' 15433614']
sorry, that ticket did not win anything
b'qg\t\x87+\x0b\xb50\n\x00\x0b\x11\xa8\xf4\x80\xeb, 526602, 8804009, 16256181, 15433614\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'


True

In [208]:
A.numbers

[13967147, 526602, 8804009, 16256181, 15433614]

In [167]:
bytes([C[i] ^ D[i] for i in range(len(C))])

b'\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10'

In [152]:
D_blc

[b'\xa6&o\x16\xa4\xf8\x0c\xdaF\x94\xed\xd0\xa4@\x1b\x8f',
 b'PO\x0c\x93xH\xbd\t[\xee\x87\x87\xb8\x1a<\x1c',
 b'\x84\xf1Q)a\x9e\xed\xb8E/\xcf\x8fT\x1a\xb8\xd4']

In [153]:
payload = b'numbers:15087615, 5622752, 7060194, 552130, 12062106'
padpay = payload + (16-len(payload)%16) * long_to_bytes(16-len(payload)%16)
print(padpay)
print()

n_blc = len(padpay)//16 + 1

C0 = b'\x00'*16
C1 = b'\x01'*16
C2 = b'\x02'*16
C3 = b'\x03'*16
C4 = b'\x04'*16
C5 = b'\x05'*16

D = b''

for byt in range(1,17):
    
    og_byt = C5[-byt]

    pb = bytes([D[i] ^ byt for i in range(len(D))][::-1])
    padbyt = C5[:len(C5)-(byt-1)] + pb 
    
    assert len(C5) == len(padbyt)

    b_correct = None
    pos_lst = []
    for b in range(256):

        trick = bytearray(padbyt)

        trick[-byt] = b

        check = A.redeemTicket(base64.b64encode( ticket + trick ))

        if check:
            pos_lst.append(b)
            
    if len(pos_lst) == 1:
        b_correct = pos_lst[0]
    elif len(pos_lst) == 2:
        b1,b2 = pos_lst
        if b1 == og_byt:
            b_correct = b2
        if b2 == og_byt:
            b_correct = b1
    else:
        print(pos_lst)
        raise ValueError('Oh oh...')
        
    D += long_to_bytes(b_correct ^ byt)
        
print(D)

b'numbers:15087615, 5622752, 7060194, 552130, 12062106\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c'

[]


ValueError: Oh oh...

In [151]:
D

b'Q*'